In [5]:
from main import load_and_preprocess_data

df, encoders = load_and_preprocess_data()

Loading and preprocessing data...
Class distribution:
is_fraud
0    1289169
1       7506
Name: count, dtype: int64
Fraud percentage: 0.58%


In [ ]:
from model import load_lstm_model
from predict import predict_next_transaction

cc_num = df['cc_num'].iloc[0]

user_transactions = df[df['cc_num'] == cc_num]

sequence_length = 10

lstm_model, scaler, encoders = load_lstm_model()

next_transaction = predict_next_transaction(
    lstm_model, user_transactions, sequence_length, scaler, df.columns, encoders
)

next_transaction

Loading model from lstm_transaction_model.h5...
Error: Model file lstm_transaction_model.h5 not found.


TypeError: cannot unpack non-iterable NoneType object